![](imgs/kodolamaczlogo.png)

# Przetwarzanie Big Data z użyciem Apache Spark

Autor notebooka: Jakub Nowacki.

## TF-IDF

Term Frequency Inverse Document Frequency (TF-IDF) jest po popularną i przydatną miarą unikalności danego terminu (słowa) dla danego dokumentu. Miara w istocie składa się z dwóch elementów, TF oraz IDF, które na końcu są ze sobą mnożone.

Term Frequency (TF), jest częstotliwością wystąpień terminu w dokumencie. Jako równanie zapisuje się go jak poniżej:

$$
TF = \frac{n_w}{n_d},
$$

gdzie: $n_w$ ile razy termin wystąpił w dokuemcie $n_d$ wszystkie słowa w dokumencie.

Z kolei Inverse Documents Frequency (IDF), zapisuje się jako:

$$
IDF = \log \left( \frac{c_d}{i_d} \right),
$$

gdzie: $c_d$ jest liczbą wszystkich dokumentów, $i_d$ jest liczbą dokumentów w których dane słowo wystąpiło. Należy pamiętać, że $\log$ jest logarytmem naturalnym przy podstawie $e$.

## Dane do analizy 

W przykładzie użyte zostaną dane z [Projektu Gutenberg](https://www.gutenberg.org). Poniższy skrypt pobierze dane jeżeli nie ma ich jeszcze w folderze `data`.

In [2]:
# IMDB dataset CERTIFICATE_VERIFY_FAILED exception workaround
import requests
requests.packages.urllib3.disable_warnings()
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context

In [3]:
import os
import urllib.request

data_path = 'data/tf-idf_docs'
if not os.path.exists(data_path):
    os.mkdir(data_path)
    
book_links = {
    'grimms_fairy_tales': 'https://www.gutenberg.org/files/2591/2591-0.txt',
    'dracula': 'http://www.gutenberg.org/cache/epub/345/pg345.txt',
    'frankenstein': 'http://www.gutenberg.org/cache/epub/84/pg84.txt',
    'moby_dick': 'http://www.gutenberg.org/files/2701/2701-0.txt',
    'tom_sawyer': 'http://www.gutenberg.org/files/74/74-0.txt',
    'war_and_peace': 'http://www.gutenberg.org/files/2600/2600-0.txt'
}

for book, link in book_links.items():
    file_name = '{}.txt'.format(book)
    file_path = os.path.join(data_path, file_name)
    if not os.path.exists(file_path):
        print('Pobieranie pliku {} do {}'.format(link, file_path))
        urllib.request.urlretrieve(link, file_path)
        print('Pobieranie {} ukończone'.format(file_path))
    else:
        print('Plik {} jest już pobrany'.format(file_path))

Pobieranie pliku https://www.gutenberg.org/files/2591/2591-0.txt do data/tf-idf_docs/grimms_fairy_tales.txt
Pobieranie data/tf-idf_docs/grimms_fairy_tales.txt ukończone
Pobieranie pliku http://www.gutenberg.org/cache/epub/345/pg345.txt do data/tf-idf_docs/dracula.txt
Pobieranie data/tf-idf_docs/dracula.txt ukończone
Pobieranie pliku http://www.gutenberg.org/cache/epub/84/pg84.txt do data/tf-idf_docs/frankenstein.txt
Pobieranie data/tf-idf_docs/frankenstein.txt ukończone
Pobieranie pliku http://www.gutenberg.org/files/2701/2701-0.txt do data/tf-idf_docs/moby_dick.txt
Pobieranie data/tf-idf_docs/moby_dick.txt ukończone
Pobieranie pliku http://www.gutenberg.org/files/74/74-0.txt do data/tf-idf_docs/tom_sawyer.txt
Pobieranie data/tf-idf_docs/tom_sawyer.txt ukończone
Pobieranie pliku http://www.gutenberg.org/files/2600/2600-0.txt do data/tf-idf_docs/war_and_peace.txt
Pobieranie data/tf-idf_docs/war_and_peace.txt ukończone


## Zadanie 

Należy policzyć miary TF-IDF dla pobranych dokumentów. Kroki do wykoniania:

1. Odcztać dane, tak aby zachować informacje z której książki pochodzi dane zdanie.
1. Podzielić zdania (linie) na słowa; słowa powinny być sprowadzone do wspólnego znaku i nie posiadać interpunkcji.
1. Policzyć miarę TF.
1. Policzyć miarę IDF.
1. Złączyć wyniki tak, żeby wynikowa tabela miała kolumny: nazwa książki, słowo (termin), TF, IDF, TF-IDF.

In [ ]:
import pyspark
import pyspark.sql.functions as func

spark = pyspark.sql.SparkSession.builder\
    .appName('tf-idf_sql')\
    .getOrCreate()

In [ ]:
books = spark.read.text(data_path)

books.show(truncate=False)

## Podpowiedzi

* Pliki mają nagłówki, ale można je zostawić i potraktować jako część dokumentu.
* Wszystko da się zrobić interfejsem Spark SQL (DataFrame lub zapytania SQL); nie ma potrzeby pisania dodatkowych funkcji.
* Wprawdzie wiemy jakie są książki, ale jest lepsza metoda dostania nazwy czytanego pliku w DataFrame; zobacz [manual Spark odnośnie funkcji](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions). 
* Generalnie, w powyższym manualu jest sporo przydatnych funkcji.